In [9]:
import pandas as pd 
import pickle

from sklearn import metrics

import seaborn as sns

# sklearn :: utils
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.externals import joblib

# sklearn :: models
#from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeClassifier

sns.set_style('whitegrid')

In [2]:
tracks = pd.read_csv('../../data/processed/tracks_processed.csv', dtype=object)
tracks.head()

,Unnamed: 0,track_id,album_date_created,album_date_released,album_favorites,album_id,album_listens,album_tags,album_title,album_tracks,...,track_interest,track_language_code,track_listens,track_number,track_title,track_year_created,bit_rate_factor,listens_factor,interest_factor,track_price
0,0,2,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,4656,en,1293,3,Food,2008,0.7,2,1.0,3.49
1,1,3,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,1470,en,514,4,Electric Ave,2008,0.7,3,0.1,0.35
2,2,5,2008-11-26 1:44,2009-01-05 0:00,4,1,6073,[],AWOL - A Way Of Life,7,...,1933,en,1151,6,This World,2008,0.7,2,0.7,2.45
3,3,10,2008-11-26 1:45,2008-02-06 0:00,4,6,47632,[],Constant Hitmaker,2,...,54881,en,50135,1,Freeway,2008,0.5,1,1.0,2.5
4,4,20,2008-11-26 1:45,2009-01-06 0:00,2,4,2710,[],Niris,13,...,978,en,361,3,Spiritual Level,2008,0.7,4,0.1,0.35


# Feature Engineering

In [3]:
X_columns=[tracks.columns.get_loc(c) for c in tracks.columns if c in ["track_listens", "track_year_created", "artist_id","bit_rate_factor"]]
y_column = tracks.columns.get_loc('interest_factor')

# Model Training

In [4]:
# split the data

threshold = 0.8
absolute_threshold = int(len(tracks)*threshold)

X_train = tracks.ix[:absolute_threshold, X_columns]
y_train = tracks.ix[:absolute_threshold, y_column]

X_test = tracks.ix[absolute_threshold:, X_columns]
y_test = tracks.ix[absolute_threshold:, y_column]

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

X_train (84720, 4)
y_train (84720,)
X_test (21180, 4)
y_test (21180,)


/home/cloudera/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [6]:
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train, y_train.values.ravel())
print ("Random Forest 10 :: ", model)

Random Forest 10 ::  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [7]:
# Dump the trained decision tree classifier with Pickle
RF_pkl_filename = 'RandomForest10-model.pkl'
# Open the file to save as pkl file
RF_model_pkl = open(RF_pkl_filename, 'wb')
pickle.dump(model, RF_model_pkl)
# Close the pickle instances
RF_model_pkl.close()

In [27]:
# load the model and predict
#["track_listens", "track_year_created", "artist_id","bit_rate_factor"]
model = joblib.load(RF_pkl_filename)
prediction = model.predict([[100000, 2008, 80, 0.5]])
print('prediction: ', prediction)

prediction:  ['1.0']
